In [3]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [9]:
df = pd.read_csv('archive/ArticlesApril2018.csv')

print('열의 개수 : ', len(df.columns))
print(df.columns)

열의 개수 :  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [11]:
df['headline'].isnull().values.any()

False

In [15]:
headline = []
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [19]:
print('총 샘플의 개수 : {}\n'.format(len(headline)))

headline = [n for n in headline if n != "Unknown"]
print('노이즈 제거 후 총 샘플의 개수 \n: {}'.format(len(headline)))
headline[:5]

총 샘플의 개수 : 1214

노이즈 제거 후 총 샘플의 개수 
: 1214


['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [20]:
def repreprocessing(s) :
    s = s.encode("utf8").decode("ascii","ignore")
    return ''.join(c for c in s if c not in punctuation).lower()

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [21]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index)+1
print('단어 집합의 크기 : %d'% vocab_size)

단어 집합의 크기 : 3494


In [24]:
sequences = list()

for line in text :
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1,len(encoded)) :
        sequence = encoded[:i+1]
        sequences.append(sequence)
sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [25]:
index_to_word = {}

for key, value in t.word_index.items() :
    index_to_word[value] = key
    
print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [26]:
max_len = max(len(l) for l in sequences)
print("샘플 최대의 길이 : {} ".format(max_len))

샘플 최대의 길이 : 24 


In [27]:
sequences = pad_sequences(sequences, maxlen= max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [29]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y= sequences[:,-1]

print(X[:3])
print(y[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]
[ 269  371 1115]


In [30]:
y = to_categorical(y, num_classes= vocab_size)

# LSTM 모델 설계

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(LSTM(128))
model.add(Dense(vocab_size,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y,epochs=200,verbose=2)

Epoch 1/200
244/244 - 3s - loss: 7.6356 - accuracy: 0.0301
Epoch 2/200
244/244 - 3s - loss: 7.1229 - accuracy: 0.0310
Epoch 3/200
244/244 - 4s - loss: 6.9837 - accuracy: 0.0309
Epoch 4/200
244/244 - 4s - loss: 6.8640 - accuracy: 0.0408
Epoch 5/200
244/244 - 4s - loss: 6.7247 - accuracy: 0.0425
Epoch 6/200
244/244 - 4s - loss: 6.5542 - accuracy: 0.0461
Epoch 7/200
244/244 - 4s - loss: 6.3571 - accuracy: 0.0514
Epoch 8/200
244/244 - 4s - loss: 6.1539 - accuracy: 0.0555
Epoch 9/200
244/244 - 4s - loss: 5.9488 - accuracy: 0.0598
Epoch 10/200
244/244 - 4s - loss: 5.7535 - accuracy: 0.0663
Epoch 11/200
244/244 - 4s - loss: 5.5695 - accuracy: 0.0689
Epoch 12/200
244/244 - 4s - loss: 5.3922 - accuracy: 0.0763
Epoch 13/200
244/244 - 4s - loss: 5.2293 - accuracy: 0.0816
Epoch 14/200
244/244 - 4s - loss: 5.0736 - accuracy: 0.0879
Epoch 15/200
244/244 - 4s - loss: 4.9219 - accuracy: 0.0971
Epoch 16/200
244/244 - 4s - loss: 4.7796 - accuracy: 0.1062
Epoch 17/200
244/244 - 4s - loss: 4.6402 - accura

244/244 - 4s - loss: 0.3396 - accuracy: 0.9166
Epoch 138/200
244/244 - 4s - loss: 0.3362 - accuracy: 0.9154
Epoch 139/200
244/244 - 4s - loss: 0.3342 - accuracy: 0.9155
Epoch 140/200
244/244 - 4s - loss: 0.3315 - accuracy: 0.9154
Epoch 141/200
244/244 - 4s - loss: 0.3300 - accuracy: 0.9164
Epoch 142/200
244/244 - 4s - loss: 0.3261 - accuracy: 0.9162
Epoch 143/200
244/244 - 4s - loss: 0.3239 - accuracy: 0.9161
Epoch 144/200
244/244 - 4s - loss: 0.3215 - accuracy: 0.9162
Epoch 145/200
244/244 - 4s - loss: 0.3188 - accuracy: 0.9171
Epoch 146/200
244/244 - 4s - loss: 0.3152 - accuracy: 0.9171
Epoch 147/200
244/244 - 4s - loss: 0.3128 - accuracy: 0.9186
Epoch 148/200
244/244 - 4s - loss: 0.3116 - accuracy: 0.9162
Epoch 149/200
244/244 - 4s - loss: 0.3134 - accuracy: 0.9154
Epoch 150/200
244/244 - 4s - loss: 0.3471 - accuracy: 0.9109
Epoch 151/200
244/244 - 4s - loss: 0.3136 - accuracy: 0.9148
Epoch 152/200
244/244 - 4s - loss: 0.3058 - accuracy: 0.9175
Epoch 153/200
244/244 - 4s - loss: 0.3

In [38]:
def sentence_generation(model, t, current_word, n ) :
    init_word = current_word
    sentence = ''
    for _ in range(n) :
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        result = model.predict_classes(encoded, verbose=0)
        
        for word, index in t.word_index.items() :
            if index == result :
                break
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word
    sentence = init_word + sentence
        
    return sentence

In [49]:
print(sentence_generation(model,t,'try',10))

try the president when a accomplished landlord gay homes of new


In [50]:
print(sentence_generation(model,t,'world',10))

world recap and change at classics dept over north accomplished of
